# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [14]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os
import re

In [15]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {
    "allpubs": {
        "file": "citations.bib",
        "venuekey": "journal",
        "venue-pretext": "",
        "collection": {
            "name": "publications",
            "permalink": "/publications/"
        }
    }
}

In [16]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [ ]:
for pubsource in publist:
    parser = bibtex.Parser()
    bibdata = parser.parse_file(publist[pubsource]["file"])

    #loop through the individual references in a given bibtex file
    for bib_id in bibdata.entries:
        #reset default date
        pub_year = "1900"
        pub_month = "01"
        pub_day = "01"
        
        b = bibdata.entries[bib_id].fields
        
        try:
            pub_year = b.get("year", b.get("date", "1900"))

            #todo: this hack for month and day needs some cleanup
            if "month" in b.keys(): 
                if(len(b["month"])<3):
                    pub_month = "0"+b["month"]
                    pub_month = pub_month[-2:]
                elif(b["month"] not in range(12)):
                    tmnth = strptime(b["month"][:3],'%b').tm_mon   
                    pub_month = "{:02d}".format(tmnth) 
                else:
                    pub_month = str(b["month"])
            if "day" in b.keys(): 
                pub_day = str(b["day"])

                
            pub_date = pub_year+"-"+pub_month+"-"+pub_day
            
            #strip out {} as needed (some bibtex entries that maintain formatting)
            clean_title = b["title"].replace("{", "").replace("}","").replace("\\","").replace(" ","-")    

            url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
            url_slug = url_slug.replace("--","-")

            # Windows safe: truncate if filename too long
            if len(md_filename) > 150:
                md_filename = md_filename[:150] + ".md"

            output_dir = os.path.abspath("../_publications/")
            os.makedirs(output_dir, exist_ok=True)
            save_path = os.path.join(output_dir, md_filename)

            with open(save_path, 'w', encoding="utf-8") as f:
                f.write(md)

            #Build Citation from text
            citation = ""

            #citation authors - todo - add highlighting for primary author?
            for author in bibdata.entries[bib_id].persons["author"]:
                fname = author.first_names[0] if author.first_names else ""
                lname = author.last_names[0] if author.last_names else ""
                citation = citation + " " + fname + " " + lname + ", "

            #citation title
            citation = citation + "\"" + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + ".\""

            ## venue fallback (try journal, booktitle, publisher, else Unknown)
            if "journal" in b:
                venue = b["journal"].replace("{", "").replace("}", "").replace("\\", "")
            elif "booktitle" in b:
                venue = b["booktitle"].replace("{", "").replace("}", "").replace("\\", "")
            elif "publisher" in b:
                venue = b["publisher"].replace("{", "").replace("}", "").replace("\\", "")
            else:
                venue = "Unknown Venue"

            citation = citation + " " + html_escape(venue)
            citation = citation + ", " + pub_year + "."

            
            ## YAML variables
            md = "---\ntitle: \""   + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + '"\n'
            
            md += """collection: """ +  publist[pubsource]["collection"]["name"]

            md += """\npermalink: """ + publist[pubsource]["collection"]["permalink"]  + html_filename
            
            note = False
            if "note" in b.keys():
                if len(str(b["note"])) > 5:
                    md += "\nexcerpt: '" + html_escape(b["note"]) + "'"
                    note = True

            md += "\ndate: " + str(pub_date) 

            md += "\nvenue: '" + html_escape(venue) + "'"
            
            url = False
            if "url" in b.keys():
                if len(str(b["url"])) > 5:
                    md += "\npaperurl: '" + b["url"] + "'"
                    url = True

            md += "\ncitation: '" + html_escape(citation) + "'"

            md += "\n---"

            
            ## Markdown description for individual page
            if note:
                md += "\n" + html_escape(b["note"]) + "\n"

            if url:
                md += "\n[Access paper here](" + b["url"] + "){:target=\"_blank\"}\n" 
            else:
                md += "\nUse [Google Scholar](https://scholar.google.com/scholar?q="+html.escape(clean_title.replace("-","+"))+"){:target=\"_blank\"} for full citation"

            md_filename = os.path.basename(md_filename)

            with open("../_publications/" + md_filename, 'w', encoding="utf-8") as f:
                f.write(md)
            print(f'SUCESSFULLY PARSED {bib_id}: \"', b["title"][:60],"..."*(len(b['title'])>60),"\"")
        # field may not exist for a reference
        except KeyError as e:
            print(f'WARNING Missing Expected Field {e} from entry {bib_id}: \"', b["title"][:30],"..."*(len(b['title'])>30),"\"")
            continue


WARNING Missing Expected Field 'journal' from entry arshad2024nitrogen: " Nitrogen Fertility Management  ... "
SUCESSFULLY PARSED yufei2022empirical: " An empirical study of the key factors affecting herders' pur ... "
SUCESSFULLY PARSED lin2024impact: " The impact of rural e-commerce participation on farmers’ ent ... "
SUCESSFULLY PARSED guo2022farmers: " Farmers’ willingness to pay a high premium for different typ ... "
SUCESSFULLY PARSED liu2023uncovering: " Uncovering the key factors influencing sustainable green pro ... "
SUCESSFULLY PARSED shi2023utilizing: " Utilizing AI models to optimize blended teaching effectivene ... "
SUCESSFULLY PARSED yufei2024research: " A Research Paradigm of Weather Index Insurance for Grassland ... "
SUCESSFULLY PARSED arshad2021effect: " The effect of climate change on cotton productivity-an empir ... "
SUCESSFULLY PARSED lin2022measurement: " Measurement and identification of relative poverty level of  ... "
SUCESSFULLY PARSED guo2023effects: " Eff